# Data Exploration Tutorial: DuckDB & Pandas

This notebook demonstrates how to explore the AIDA Challenge database using DuckDB and pandas. We'll focus on the staging layer which contains the raw data imported from CSV files.

## Index
- Connecting to a DuckDB database
- Listing available tables and schemas
- Querying data with DuckDB SQL
- Converting DuckDB results to pandas DataFrames
- Basic data exploration techniques
- Understanding the staging layer data structure

In [ ]:
# Import required libraries
import duckdb
import pandas as pd
from pathlib import Path

# Set pandas display options for better readability
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)

## 1. Connecting to DuckDB

DuckDB is an embedded analytical database that's perfect for data analysis. It stores data in a single file and provides excellent performance for analytical queries.

**Best Practice:** Always use a context manager or remember to close the connection when done!

In [ ]:
# Connect to DuckDB database
# Path is relative to notebook location
db_path = Path("data/aida_challenge.duckdb").absolute()
con = duckdb.connect(db_path)

print(f"✓ Connected to database")

## 2. Exploring Database Structure

Let's start by understanding what schemas and tables are available in our database.

In [ ]:
# List all schemas
schemas = con.execute(
    "SELECT schema_name FROM information_schema.schemata ORDER BY schema_name"
).df()
print("Available schemas:")
print(schemas)

In [ ]:
# List all tables in the staging schema
tables = con.execute(
    """
    SELECT table_schema, table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'main_staging'
    ORDER BY table_name
"""
).df()

print("Staging Layer Tables:")
print(tables)

### Optional: Launch DuckDB UI

DuckDB provides a web UI for visual exploration. Uncomment and run the next cell to launch it.

In [ ]:
# Uncomment to start the DuckDB UI
# con.execute("CALL start_ui()")

## 3. Exploring Staging Tables

Now let's explore each table in the staging layer. We'll look at:
- Table structure (columns and types)
- Row counts
- Sample data
- Basic statistics

### 3.1 Clients (stg_clienti)

This table contains customer information.

In [ ]:
# Method 1: Using con.execute() and converting to DataFrame
clients = con.execute("SELECT * FROM aida_challenge.main_staging.stg_clienti")
clients_df = clients.df()

print(f"Total clients: {len(clients_df)}")
print(f"\nColumns: {clients_df.columns.tolist()}")
print(f"\nData types:\n{clients_df.dtypes}")

# Display sample records
print("\nSample records:")
clients_df.head()

In [ ]:
# Method 2: DuckDB can query pandas DataFrames directly!
# This is very powerful for combining SQL queries with pandas data

# Query the DataFrame we just created
result = duckdb.sql(
    """
    SELECT 
        COUNT(*) as total_clients,
        COUNT(DISTINCT codice_cliente) as unique_clients
    FROM clients_df
"""
).df()

print("Client Statistics:")
result

### 3.2 Policies (stg_polizze)

Insurance policies data.

In [ ]:
# Get policies data
policies_df = con.execute("SELECT * FROM aida_challenge.main_staging.stg_polizze").df()

print(f"Total policies: {len(policies_df)}")
print(f"\nColumns: {policies_df.columns.tolist()}")
print("\nSample data:")
policies_df.head()

In [ ]:
# Analyze policy types using DuckDB SQL
policy_types = con.execute(
    """
    SELECT 
        prodotto,
        area_bisogno,
        COUNT(*) as count,
        ROUND(AVG(premio_totale_annuo), 2) as avg_annual_premium,
        ROUND(AVG(massimale), 2) as avg_insured_value
    FROM aida_challenge.main_staging.stg_polizze
    GROUP BY prodotto, area_bisogno
    ORDER BY count DESC
"""
).df()

print("Policy Types Analysis:")
policy_types

### 3.3 Claims (stg_sinistri)

Data about insurance claims/incidents.

In [ ]:
# Get claims data
claims_df = con.execute("SELECT * FROM aida_challenge.main_staging.stg_sinistri").df()

print(f"Total claims: {len(claims_df)}")
print(f"\nColumns: {claims_df.columns.tolist()}")
print("\nSample data:")
claims_df.head()

In [ ]:
# Analyze claims by product and status
claims_analysis = con.execute(
    """
    SELECT 
        prodotto,
        area_bisogno,
        stato_liquidazione,
        COUNT(*) as count,
        ROUND(AVG(importo_liquidato), 2) as avg_paid
    FROM aida_challenge.main_staging.stg_sinistri
    GROUP BY prodotto, area_bisogno, stato_liquidazione
    ORDER BY prodotto, area_bisogno
"""
).df()

print("Claims Analysis by Product and Status:")
claims_analysis

### 3.4 Complaints (stg_reclami)

Customer complaints data.

In [ ]:
# Get complaints data
complaints_df = con.execute("SELECT * FROM aida_challenge.main_staging.stg_reclami").df()

print(f"Total complaints: {len(complaints_df)}")
print(f"\nColumns: {complaints_df.columns.tolist()}")
print("\nSample data:")
complaints_df.head()

In [ ]:
# Analyze complaints by product and area
complaints_analysis = con.execute(
    """
    SELECT 
        prodotto,
        area_bisogno,
        COUNT(*) as count,
        COUNT(DISTINCT codice_cliente) as unique_clients
    FROM aida_challenge.main_staging.stg_reclami
    GROUP BY prodotto, area_bisogno
    ORDER BY count DESC
"""
).df()

print("Complaints Analysis:")
complaints_analysis

### 3.5 Customer Interactions (stg_interazioni_clienti)

Records of customer interactions with the company.

In [ ]:
# Get interactions data
interactions_df = con.execute(
    "SELECT * FROM aida_challenge.main_staging.stg_interazioni_clienti"
).df()

print(f"Total interactions: {len(interactions_df)}")
print(f"\nColumns: {interactions_df.columns.tolist()}")
print("\nSample data:")
interactions_df.head()

In [ ]:
# Analyze interactions by type and outcome
interactions_analysis = con.execute(
    """
    SELECT 
        tipo_interazione,
        esito,
        COUNT(*) as count,
        ROUND(AVG(durata_minuti), 1) as avg_duration_minutes,
        SUM(CASE WHEN CAST(conversione AS VARCHAR) = 'Sì' THEN 1 ELSE 0 END) as conversions
    FROM aida_challenge.main_staging.stg_interazioni_clienti
    GROUP BY tipo_interazione, esito
    ORDER BY count DESC
"""
).df()

print("Interactions Analysis:")
interactions_analysis

### 3.6 Properties (stg_abitazioni)

Information about insured properties.

In [ ]:
# Get properties data
properties_df = con.execute("SELECT * FROM aida_challenge.main_staging.stg_abitazioni").df()

print(f"Total properties: {len(properties_df)}")
print(f"\nColumns: {properties_df.columns.tolist()}")
print("\nSample data:")
properties_df.head()

In [ ]:
# Analyze properties by location
properties_analysis = con.execute(
    """
    SELECT 
        luogo_residenza,
        COUNT(*) as count,
        ROUND(AVG(metratura), 1) as avg_square_meters,
        SUM(CASE WHEN CAST(sistema_allarme AS VARCHAR) = 'Sì' THEN 1 ELSE 0 END) as with_alarm
    FROM aida_challenge.main_staging.stg_abitazioni
    GROUP BY luogo_residenza
    ORDER BY count DESC
"""
).df()

print("Properties Analysis:")
properties_analysis.head(10)

### 3.7 Competitor Products (stg_competitor_prodotti)

Information about competitor insurance products.

In [ ]:
# Get competitor products data
competitor_df = con.execute(
    "SELECT * FROM aida_challenge.main_staging.stg_competitor_prodotti"
).df()

print(f"Total competitor products: {len(competitor_df)}")
print(f"\nColumns: {competitor_df.columns.tolist()}")
print("\nSample data:")
competitor_df.head()

In [ ]:
# Analyze competitor products
competitor_analysis = con.execute(
    """
    SELECT 
        Competitor,
        Tipo_Prodotto,
        ROUND(AVG(Premio_Medio), 2) as avg_premium,
        ROUND(AVG(Massimale_Medio), 2) as avg_coverage,
        ROUND(AVG(Rating_Clienti), 2) as avg_rating
    FROM aida_challenge.main_staging.stg_competitor_prodotti
    GROUP BY Competitor, Tipo_Prodotto
    ORDER BY Competitor, Tipo_Prodotto
"""
).df()

print("Competitor Products Analysis:")
competitor_analysis

## 4. Advanced DuckDB & Pandas Integration

DuckDB provides seamless integration with pandas. Here are some powerful techniques:

In [ ]:
# Example 1: Join DataFrames using SQL
# Let's join clients with their policies

join_result = duckdb.sql(
    """
    SELECT 
        c.codice_cliente,
        c.nome,
        c.cognome,
        p.prodotto,
        p.area_bisogno,
        p.premio_totale_annuo,
        p.data_emissione,
        p.data_scadenza
    FROM clients_df c
    JOIN policies_df p ON c.codice_cliente = p.codice_cliente
    LIMIT 10
"""
).df()

print("Clients with their policies:")
join_result

In [ ]:
# Example 2: Complex aggregation
# Find clients with the most claims

high_claims_clients = duckdb.sql(
    """
    SELECT 
        c.codice_cliente,
        c.nome,
        c.cognome,
        COUNT(cl.sinistro) as total_claims,
        ROUND(SUM(cl.importo_liquidato), 2) as total_paid
    FROM clients_df c
    JOIN claims_df cl ON c.codice_cliente = cl.codice_cliente
    GROUP BY c.codice_cliente, c.nome, c.cognome
    HAVING COUNT(cl.sinistro) > 1
    ORDER BY total_claims DESC
    LIMIT 10
"""
).df()

print("Clients with multiple claims:")
high_claims_clients

In [ ]:
# Example 3: Window functions
# Calculate running totals and rankings

policy_trends = duckdb.sql(
    """
    SELECT 
        data_emissione,
        prodotto,
        COUNT(*) as policies_started,
        SUM(COUNT(*)) OVER (
            PARTITION BY prodotto 
            ORDER BY data_emissione
        ) as cumulative_policies
    FROM policies_df
    GROUP BY data_emissione, prodotto
    ORDER BY prodotto, data_emissione
    LIMIT 20
"""
).df()

print("Policy trends with running totals:")
policy_trends

## 5. Data Quality Checks

Let's perform some basic data quality checks on our staging data.

In [ ]:
# Check for NULL values in each table
tables_to_check = [
    ("clients_df", clients_df),
    ("policies_df", policies_df),
    ("claims_df", claims_df),
    ("complaints_df", complaints_df),
    ("interactions_df", interactions_df),
    ("properties_df", properties_df),
    ("competitor_df", competitor_df),
]

print("NULL Values Summary:\n")
for table_name, df in tables_to_check:
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"{table_name}:")
        print(null_counts[null_counts > 0])
        print()

In [ ]:
# Check for duplicate records
print("Duplicate Checks:\n")
for table_name, df in tables_to_check:
    duplicates = df.duplicated().sum()
    print(f"{table_name}: {duplicates} duplicate rows")

## 6. Summary Statistics

Let's create a comprehensive overview of all staging tables.

In [ ]:
# Create summary of all tables
summary_data = []

for table_name, df in tables_to_check:
    summary_data.append(
        {
            "Table": table_name.replace("_df", ""),
            "Rows": len(df),
            "Columns": len(df.columns),
            "Memory (MB)": round(df.memory_usage(deep=True).sum() / 1024 / 1024, 2),
        }
    )

summary_df = pd.DataFrame(summary_data)
print("Staging Layer Overview:")
summary_df

## 7. Key Takeaways

**DuckDB Benefits:**
- Fast analytical queries on large datasets
- SQL interface that's familiar and powerful
- Seamless pandas integration
- In-process database (no server needed)
- Efficient storage and query execution

**Best Practices:**
1. Use DuckDB for complex queries and aggregations
2. Convert to pandas DataFrames for final manipulation and visualization
3. DuckDB can query pandas DataFrames directly - very powerful!
4. Always close connections or use context managers
5. Use window functions for advanced analytics

**Next Steps:**
- Explore the intermediate and marts layers
- Create visualizations of key metrics
- Perform deeper analysis on specific business questions

In [ ]:
# Clean up: Close the database connection
con.close()
print("✓ Database connection closed")